In [1]:
import math
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd


In [2]:
X = np.array([[2.7810836,2.550537003],
	[1.465489372,2.362125076],
	[3.396561688,4.400293529],
	[1.38807019,1.850220317],
	[3.06407232,3.005305973],
	[7.627531214,2.759262235],
	[5.332441248,2.088626775],
	[6.922596716,1.77106367],
	[8.675418651,-0.242068655],
	[7.673756466,3.508563011]])

Y = np.array([0,0,0,0,0,1,1,1,1,1])

X = X.reshape(X.shape[0], X.shape[1],1)
Y = Y.reshape(Y.shape[0], 1)

X_val=X
Y_val=Y

In [3]:
print(X.shape, Y.shape)

(10, 2, 1) (10, 1)


In [17]:
#RNN Architecture

learning_rate = 0.05
nepoch = 500               
T = 2                   # length of sequence
hidden_dim = 10         
output_dim = 1

# bptt_truncate = 0
# min_clip_value = -10
# max_clip_value = 10

In [18]:
#randomly initialize weights

U = np.random.uniform(0, 1, (hidden_dim, T))
W = np.random.uniform(0, 1, (hidden_dim, hidden_dim))
V = np.random.uniform(0, 1, (output_dim, hidden_dim))
print(U,V,W)

[[0.72671086 0.0265279 ]
 [0.39711414 0.66179513]
 [0.89960486 0.82030839]
 [0.12833293 0.01927782]
 [0.38879306 0.47535484]
 [0.16887966 0.78985779]
 [0.00927101 0.92719376]
 [0.4170722  0.5742529 ]
 [0.18466165 0.72607017]
 [0.93470993 0.7891249 ]] [[0.66133693 0.77120423 0.79695688 0.68563813 0.21805199 0.74892262
  0.1706146  0.0151264  0.64303345 0.06579108]] [[0.98329566 0.98557831 0.32609062 0.1120066  0.8294791  0.89507744
  0.25658782 0.0517692  0.54561896 0.48531698]
 [0.67802044 0.54407174 0.87120095 0.06705982 0.14458609 0.2097538
  0.14910333 0.05336298 0.87370592 0.32163788]
 [0.770716   0.27657976 0.1030236  0.37131522 0.65072275 0.30508242
  0.09860316 0.36767594 0.46121393 0.68712096]
 [0.3544175  0.13329869 0.27861017 0.54255119 0.7908065  0.58265572
  0.22036354 0.16028004 0.44997848 0.61886615]
 [0.09981604 0.9795927  0.87715666 0.39392445 0.58424869 0.22136064
  0.65308954 0.03738507 0.03608689 0.98201116]
 [0.12997746 0.73165674 0.02039179 0.01558662 0.45652142 0.

In [13]:
#forward pass
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

for epoch in range(nepoch):
    # check loss on train
    loss = 0.0
    
    # do a forward pass to get prediction
    for i in range(Y.shape[0]):
        x, y = X[i], Y[i]                    # get input, output values of each record
        prev_s = np.zeros((hidden_dim, 1))   # here, prev-s is the value of the previous activation of hidden layer; which is initialized as all zeroes
        for t in range(T):
            new_input = np.zeros(x.shape)    # we then do a forward pass for every timestep in the sequence
            new_input[t] = x[t]              # for this, we define a single input for that timestep
            mulu = np.dot(U, new_input)
            mulw = np.dot(W, prev_s)
            add = mulw + mulu
            s = sigmoid(add)
            mulv = sigmoid(np.dot(V, s))
            prev_s = s

    # calculate error 
        loss_per_record = (y - mulv)**2 / 2
        loss += loss_per_record
    loss = loss / float(y.shape[0])

    # # check loss on val
    # val_loss = 0.0
    # for i in range(Y_val.shape[0]):
    #     x, y = X_val[i], Y_val[i]
    #     prev_s = np.zeros((hidden_dim, 1))
    #     for t in range(T):
    #         new_input = np.zeros(x.shape)
    #         new_input[t] = x[t]
    #         mulu = np.dot(U, new_input)
    #         mulw = np.dot(W, prev_s)
    #         add = mulw + mulu
    #         s = sigmoid(add)
    #         mulv = sigmoid(np.dot(V, s))
    #         prev_s = s

    #     loss_per_record = (y - mulv)**2 / 2
    #     val_loss += loss_per_record
    # val_loss = val_loss / float(y.shape[0])

    print('Epoch: ', epoch + 1, ', Loss: ', loss)

    # train model
    for i in range(Y.shape[0]):
        x, y = X[i], Y[i]
    
        layers = []
        prev_s = np.zeros((hidden_dim, 1))
        dU = np.zeros(U.shape)
        dV = np.zeros(V.shape)
        dW = np.zeros(W.shape)
        
        dU_t = np.zeros(U.shape)
        dV_t = np.zeros(V.shape)
        dW_t = np.zeros(W.shape)
        
        dU_i = np.zeros(U.shape)
        dW_i = np.zeros(W.shape)
        
        # forward pass
        for t in range(T):
            new_input = np.zeros(x.shape)
            new_input[t] = x[t]
            mulu = np.dot(U, new_input)
            mulw = np.dot(W, prev_s)
            add = mulw + mulu
            s = sigmoid(add)
            mulv = sigmoid(np.dot(V, s))
            layers.append({'s':s, 'prev_s':prev_s})
            prev_s = s

        # derivative of pred
        dmulv = (mulv - y)*(1-mulv)*(mulv)
        
        # backward pass
        for t in range(T):
            dV_t = np.dot(dmulv, np.transpose(layers[t]['s']))
            dsv = np.dot(np.transpose(V), dmulv)
            
            ds = dsv
            dadd = add * (1 - add) * ds
            
            dmulw = dadd * np.ones_like(mulw)

            dprev_s = np.dot(np.transpose(W), dmulw)


            for i in range(t, 0, -1):
                ds = dsv + dprev_s
                dadd = add * (1 - add) * ds

                dmulw = dadd * np.ones_like(mulw)
                dmulu = dadd * np.ones_like(mulu)

                dW_i = np.dot(W, layers[t]['prev_s'])
                dprev_s = np.dot(np.transpose(W), dmulw)

                new_input = np.zeros(x.shape)
                new_input[t] = x[t]
                dU_i = np.dot(U, new_input)
                dx = np.dot(np.transpose(U), dmulu)

                dU_t += dU_i
                dW_t += dW_i
                
            dV += dV_t
            dU += dU_t
            dW += dW_t

            # if dU.max() > max_clip_value:
            #     dU[dU > max_clip_value] = max_clip_value
            # if dV.max() > max_clip_value:
            #     dV[dV > max_clip_value] = max_clip_value
            # if dW.max() > max_clip_value:
            #     dW[dW > max_clip_value] = max_clip_value
                
            
            # if dU.min() < min_clip_value:
            #     dU[dU < min_clip_value] = min_clip_value
            # if dV.min() < min_clip_value:
            #     dV[dV < min_clip_value] = min_clip_value
            # if dW.min() < min_clip_value:
            #     dW[dW < min_clip_value] = min_clip_value
        
        # update
        U -= learning_rate * dU
        V -= learning_rate * dV
        W -= learning_rate * dW

Epoch:  1 , Loss:  [[1.34203838]]
Epoch:  2 , Loss:  [[1.29616776]]
Epoch:  3 , Loss:  [[1.2830914]]
Epoch:  4 , Loss:  [[1.2789503]]
Epoch:  5 , Loss:  [[1.27711353]]
Epoch:  6 , Loss:  [[1.27592767]]
Epoch:  7 , Loss:  [[1.27495726]]
Epoch:  8 , Loss:  [[1.27408267]]
Epoch:  9 , Loss:  [[1.27326892]]
Epoch:  10 , Loss:  [[1.27250425]]
Epoch:  11 , Loss:  [[1.2717834]]
Epoch:  12 , Loss:  [[1.27110309]]
Epoch:  13 , Loss:  [[1.27046068]]
Epoch:  14 , Loss:  [[1.26985382]]
Epoch:  15 , Loss:  [[1.26928037]]
Epoch:  16 , Loss:  [[1.26873833]]
Epoch:  17 , Loss:  [[1.2682258]]
Epoch:  18 , Loss:  [[1.26774106]]
Epoch:  19 , Loss:  [[1.26728244]]
Epoch:  20 , Loss:  [[1.26684841]]
Epoch:  21 , Loss:  [[1.26643753]]
Epoch:  22 , Loss:  [[1.26604846]]
Epoch:  23 , Loss:  [[1.26567991]]
Epoch:  24 , Loss:  [[1.26533072]]
Epoch:  25 , Loss:  [[1.26499976]]
Epoch:  26 , Loss:  [[1.26468599]]
Epoch:  27 , Loss:  [[1.26438844]]
Epoch:  28 , Loss:  [[1.2641062]]
Epoch:  29 , Loss:  [[1.26383839]]

In [19]:
preds_raw = []
preds = []
for i in range(Y.shape[0]):
    x, y = X[i], Y[i]
    prev_s = np.zeros((hidden_dim, 1))
    # Forward pass
    for t in range(T):
        mulu = np.dot(U, x)
        mulw = np.dot(W, prev_s)
        add = mulw + mulu
        s = sigmoid(add)
        mulv = sigmoid(np.dot(V, s))
        prev_s = s

    preds_raw.append(mulv)
    preds.append(np.round(mulv))
    
preds = np.array(preds)

In [20]:
preds_raw

[array([[0.99150841]]),
 array([[0.99141356]]),
 array([[0.99156588]]),
 array([[0.99132975]]),
 array([[0.99153487]]),
 array([[0.99159781]]),
 array([[0.99155818]]),
 array([[0.99157423]]),
 array([[0.99148384]]),
 array([[0.99160367]])]

In [21]:
result  = pd.DataFrame((Y.reshape(Y.shape[0],).tolist(),preds.reshape(preds.shape[0],).tolist())).T
result.columns  = ("actual", "preds")
conf_matrix  =  pd.crosstab(result.actual, result.preds)
conf_matrix

preds,1.0
actual,
0.0,5
1.0,5
